# Multiple dispatch

In this notebook we'll explore **multiple dispatch**, which is a key feature of Julia.

Multiple dispatch makes software *generic* and *fast*!

#### Starting with the familiar

To understand multiple dispatch in Julia, let's start with what we've already seen.

We can declare functions in Julia without giving Julia any information about the types of the input arguments that function will receive:

In [1]:
f(x) = x^2

f (generic function with 1 method)

and then Julia will determine on its own which input argument types make sense and which do not:

In [2]:
f(10)

100

In [3]:
f([1, 2, 3])

LoadError: MethodError: no method matching ^(::Array{Int64,1}, ::Int64)
Closest candidates are:
  ^(!Matched::BigInt, ::Integer) at gmp.jl:602
  ^(!Matched::Regex, ::Integer) at regex.jl:729
  ^(!Matched::Missing, ::Integer) at missing.jl:155
  ...

#### Specifying the types of our input arguments

However, we also have the *option* to tell Julia explicitly what types our input arguments are allowed to have.

For example, let's write a function `foo` that only takes strings as inputs.

In [4]:
foo(x::String, y::String) = println("My inputs x and y are both strings!")

foo (generic function with 1 method)

We see here that in order to restrict the type of `x` and `y` to `String`s, we just follow the input argument name by a double colon and the keyword `String`.

Now we'll see that `foo` works on `String`s and doesn't work on other input argument types.

In [5]:
foo("hello", "hi!")

My inputs x and y are both strings!


In [6]:
foo(3, 4)

LoadError: MethodError: no method matching foo(::Int64, ::Int64)

In [7]:
methods(foo)

# 1 method for generic function "foo":
[1] foo(x::String, y::String) in Main at In[4]:1

To get `foo` to work on integer (`Int`) inputs, let's tack `::Int` onto our input arguments when we declare `foo`.

In [8]:
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")

foo (generic function with 2 methods)

In [9]:
methods(foo)

# 2 methods for generic function "foo":
[1] foo(x::Int64, y::Int64) in Main at In[8]:1
[2] foo(x::String, y::String) in Main at In[4]:1

In [10]:
foo(3, 4)

My inputs x and y are both integers!


Now `foo` works on integers! But look, `foo` also still works when `x` and `y` are strings!

In [11]:
foo("hello", "hi!")

My inputs x and y are both strings!


This is starting to get to the heart of multiple dispatch. When we declared

```julia
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")
```
we didn't overwrite or replace
```julia
foo(y::String, y::String)```

Instead, we just added an additional ***method*** to the ***generic function*** called `foo`.

A ***generic function*** is the abstract concept associated with a particular operation.

For example, the generic function `+` represents the concept of addition.

A ***method*** is a specific implementation of a generic function for *particular argument types*.

For example, `+` has methods that accept floating point numbers, integers, matrices, etc.

We can use the `methods` to see how many methods there are for `foo`.

In [12]:
methods(foo)

# 2 methods for generic function "foo":
[1] foo(x::Int64, y::Int64) in Main at In[8]:1
[2] foo(x::String, y::String) in Main at In[4]:1

In [13]:
methods(+)

# 184 methods for generic function "+":
[1] +(B::BitArray{2}, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/LinearAlgebra/src/uniformscaling.jl:117
[2] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:399
[3] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:518
[4] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:517
[5] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:516
[6] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:476
[7] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:524
[8] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:530
[9] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:116
[10] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/periods.jl:347
[11] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/periods.jl:345
[12] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/periods.jl:375
[13] +(::Missing, ::Missing) in Base at missing.jl:114
[14] +(::Missing) in Base at missing.jl:100
[15] +(::Missing, ::Number) in Base at missing.jl:115
[16] +(a::Pkg.Resolve.VersionWeight, b::Pkg.Resolve.VersionWeight) in Pkg.Resolve at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Pkg/src/Resolve/versionweights.jl:22
[17] +(a::Pkg.Resolve.FieldValue, b::Pkg.Resolve.FieldValue) in Pkg.Resolve at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Pkg/src/Resolve/fieldvalues.jl:43
[18] +(a::Float16, b::Float16) in Base at float.jl:398
[19] +(x::Float64, y::Float64) in Base at float.jl:401
[20] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:286
[21] +(x::Bool, y::Bool) in Base at bool.jl:96
[22] +(x::Bool) in Base at bool.jl:93
[23] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:103
[24] +(x::Bool, z::Complex) in Base at complex.jl:293
[25] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:548
[26] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:541
[27] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:535
[28] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:394
[29] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:363
[30] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:370
[31] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:378
[32] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:386
[33] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:287
[34] +(z::Complex{Bool}, x::Real) in Base at complex.jl:301
[35] +(t::Dates.Time, dt::Dates.Date) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:20
[36] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:78
[37] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:47
[38] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:23
[39] +(x::Dates.DateTime, y::Dates.Period) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:76
[40] +(x::Float32, y::Float32) in Base at float.jl:400
[41] +(x::Dates.Date, y::Dates.Day) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:7

So, we now can call `foo` on integers or strings. When you call `foo` on a particular set of arguments, Julia will infer the types of the inputs and dispatch the appropriate method. *This* is multiple dispatch.

Multiple dispatch makes our code generic and fast. Our code can be generic and flexible because we can write code in terms of abstract operations such as addition and multiplication, rather than in terms of specific implementations. At the same time, our code runs quickly because Julia is able to call efficient methods for the relevant types.

To see which method is being dispatched when we call a generic function, we can use the @which macro:

In [14]:
@which foo(3, 4)

foo(x::Int64, y::Int64) in Main at In[8]:1

In [15]:
@which 3.0 + 3.0

+(x::Float64, y::Float64) in Base at float.jl:401

Given that a method written specifically for floating point numbers is dispatched on `3.0 + 3.0`, the LLVM code generated is extremely terse:

In [16]:
@code_llvm 3.0 + 3.0


;  @ float.jl:401 within `+'
define double @"julia_+_1515"(double, double) {
top:
  %2 = fadd double %0, %1
  ret double %2
}


In [17]:
@code_native 3.0 + 3.0

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ float.jl:401 within `+'
	vaddsd	%xmm1, %xmm0, %xmm0
	retq
; └
; ┌ @ float.jl:401 within `<invalid>'
	nopw	%cs:(%rax,%rax)
	nop
; └


Note that Julia is fast even when we write generic function definitions because, at the end of the day, specific/tailored methods are called under the hood.

For example, note that we can declare the adding function `myadd` without providing any type annotations - 

In [18]:
myadd(x, y) = x + y

myadd (generic function with 1 method)

and though we haven't constrained the types of `x` and `y`, we'll see that the LLVM code generated for `myadd(3.0, 3.0)` looks like that of `3.0 + 3.0`

In [19]:
@code_llvm myadd(3.0, 3.0)


;  @ In[18]:1 within `myadd'
define double @julia_myadd_1580(double, double) {
top:
; ┌ @ float.jl:401 within `+'
   %2 = fadd double %0, %1
; └
  ret double %2
}


In [23]:
@code_typed myadd(3, 3.0)

CodeInfo(
1 ─ %1 = Base.sitofp(Float64, x)::Float64
│   %2 = Base.add_float(%1, y)::Float64
└──      return %2
) => Float64

In [24]:
?promote

search: promote promote_type promote_rule promote_shape pointer_from_objref



```
promote(xs...)
```

Convert all arguments to a common type, and return them all (as a tuple). If no arguments can be converted, an error is raised.

# Examples

```jldoctest
julia> promote(Int8(1), Float16(4.5), Float32(4.1))
(1.0f0, 4.5f0, 4.1f0)
```


In [26]:
typeof(1.0e-4)

Float64

In [27]:
typeof(1.0f-4)

Float32